# Extended NN

In [ ]:
import json
import os
import tensorflow as tf
from models import ExtendedNN
from datasets import ENNDataset, SGAE_dataset
from algomorphism.figure.opt import multiple_models_history_figure, pca_denoising_figure, print_confmtx
from algomorphism.method.opt import pca_denoising_preprocessing
from algomorphism.dataset.datasets.structure import FromEdgesListExamples
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# optional for GPU usage

# for gpu in tf.config.list_physical_devices('GPU'):
#     print(gpu)
#     tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
embs_id = '64'
embsD = 64
labels = ['0','1','2']
# all combinations seen (s), unseen (u):
    # {s: ['0','1'], u:['2']}, {s: ['0','2'], u:['1']}, {s: ['1','2'], u:['0']}

seen_labels = ['0','1']
unseen_labels = ['2']
outfilekey = 's-{}-u-{}'.format('-'.join(seen_labels), '-'.join(unseen_labels))
extd = ENNDataset(embs_id, '.')

In [ ]:
knn = KNeighborsClassifier(1)
r_emb = extd.r_emb_disct_preprosesing(seen_labels, unseen_labels)
X = tf.concat([r_emb[label].reshape(1,-1) for label in r_emb.keys()],axis=0)
Xall = tf.concat([r_emb[key].reshape(1,-1) for key in r_emb.keys()],axis=0)
labels_all = list(r_emb.keys())
Y = extd.get_r_onehot_from_labels([label for label in r_emb.keys()])
knn.fit(X,Y)

In [ ]:
eld = FromEdgesListExamples(SGAE_dataset['route_links'])

w_p = float(eld.a.shape[1]*eld.a.shape[2] * eld.a.shape[2] - tf.reduce_sum(eld.a)) / tf.reduce_sum(eld.a)
norm = eld.a.shape[1] * eld.a.shape[2] / float((eld.a.shape[1] * eld.a.shape[2] - tf.reduce_sum(eld.a)) * 2)
norm = tf.cast(norm, tf.float32)
w_p = tf.cast(w_p, tf.float32)
a_dim = eld.a.shape[1]
lamda = 1e-2

enn = ExtendedNN(extd, embsD,[a_dim,64], w_p=w_p, norm=norm, knn=knn,learning_rate=1e-1, early_stop_vars=None, lamda=lamda)

In [ ]:
enn.train(extd, 100, print_types=['train', 'val'])

In [ ]:
save_history_to_file = False
if save_history_to_file:
    if not os.path.exists('./data/extnn/'):
        os.makedirs('./data/extnn/')
    with open('./data/extnn/history-{}.json'.format(outfilekey), 'w') as outfile:
        json.dump(enn.history, outfile, indent=4)

In [ ]:
save_obj = ['.','indoor_localization','extnn', 'cost-score', outfilekey]
multiple_models_history_figure([enn], save_obj=None)

In [ ]:
val_seen = enn.score_mtr.metric_dataset(extd.val.seen)
val_unseen = enn.score_mtr.metric_dataset(extd.val.unseen)

print('val seen', val_seen)
print('val unseen', val_unseen)
print('HM', 2*val_seen*val_unseen/(val_seen + val_unseen))


print_confmtx(enn, extd, outfilekey+'_r_', -2, -2)
print_confmtx(enn, extd, outfilekey+'_a_', -1, -1)

In [ ]:
pca_dict, pca_emb, knn_pca= pca_denoising_preprocessing(enn, extd, X, Y, pca_emb_idxs=[0, 1, 2], emb_idx=-2 ,example_predicted_types=['val_seen', 'val_unseen'])
save_obj = ['.','indoor_localization','extnn', 'node-emb', outfilekey]
pca_denoising_figure(pca_dict, pca_emb, knn_pca, r_emb.keys(), save_obj=None, pca_emb_idxs=[0,1,2])